# African Wildlife Image Analysis
**Computer Vision Coursework - Group 1**

This notebook covers:
- **Part A:** OpenCV Image Processing (Filtering, Edge Detection, Contours)
- **Part B:** Deep Learning Classification using MobileNetV2

---

## 🔧 Setup Section
### Install Required Packages

In [ ]:
# Install kaggle API for dataset download
!pip install -q kaggle

# Import necessary libraries
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import files
from google.colab import drive
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
from pathlib import Path
import json
import shutil

print("✅ All libraries imported successfully!")
print(f"TensorFlow Version: {tf.__version__}")
print(f"OpenCV Version: {cv2.__version__}")

### Check GPU Availability

In [ ]:
# Check if GPU is available
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    print("🚀 GPU Available:", gpu_devices[0].name)
    print("Training will be FAST!")
else:
    print("⚠️ No GPU detected. Training will be slower.")
    print("To enable GPU: Runtime → Change runtime type → T4 GPU")

### Mount Google Drive (to save results)

In [ ]:
# Mount Google Drive to save models and results
drive.mount('/content/drive')

# Create a folder for this project
project_folder = '/content/drive/MyDrive/african_wildlife_project'
os.makedirs(project_folder, exist_ok=True)
print(f"✅ Project folder created: {project_folder}")

### Upload Kaggle API Key
**⚠️ ACTION REQUIRED:** Click the "Choose Files" button below and upload your `kaggle.json` file

In [ ]:
# Upload your kaggle.json file
print("📤 Please upload your kaggle.json file:")
uploaded = files.upload()

# Set up Kaggle API credentials
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

print("✅ Kaggle API configured successfully!")

### Download African Wildlife Dataset

In [ ]:
# Download the dataset from Kaggle
print("⬇️ Downloading African Wildlife dataset...")
!kaggle datasets download -d biancaferreira/african-wildlife

# Unzip the dataset
print("📦 Extracting dataset...")
!unzip -q african-wildlife.zip -d /content/african_wildlife

print("✅ Dataset downloaded and extracted!")

### Explore Dataset Structure

In [ ]:
# Explore the dataset structure
dataset_path = '/content/african_wildlife'

# Find all subdirectories (animal classes)
def explore_dataset(path):
    print("📁 Dataset Structure:")
    print("=" * 50)
    
    # Look for common dataset structures
    possible_dirs = [path, os.path.join(path, 'train'), os.path.join(path, 'dataset')]
    
    found_dir = None
    for base_dir in possible_dirs:
        if os.path.exists(base_dir):
            items = os.listdir(base_dir)
            print(f"\nContents of {base_dir}:")
            subdirs_with_images = []
            
            for item in items:
                item_path = os.path.join(base_dir, item)
                if os.path.isdir(item_path):
                    # Count images in this directory
                    image_files = [f for f in os.listdir(item_path) 
                                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                    if image_files:
                        subdirs_with_images.append(item)
                        print(f"  📂 {item}: {len(image_files)} images")
            
            # If this directory has subdirectories with images, use it
            if subdirs_with_images and found_dir is None:
                found_dir = base_dir
    
    # Return the directory that has subdirectories with images, or default to the input path
    return found_dir if found_dir else path

base_dir = explore_dataset(dataset_path)

### Select Animal Classes for Analysis
**⚠️ DECISION POINT:** Based on the structure above, we'll select 4 animal classes

In [ ]:
# Define the correct data path based on exploration
# Use the base_dir from the previous cell
data_dir = base_dir  # Use the detected directory from explore_dataset

# List all available classes
all_classes = [d for d in os.listdir(data_dir) 
               if os.path.isdir(os.path.join(data_dir, d)) and not d.startswith('.')]

print("Available animal classes:")
for i, cls in enumerate(all_classes, 1):
    print(f"{i}. {cls}")

# Select 4 classes with good representation
# You can manually change these based on what you see above
if len(all_classes) >= 4:
    selected_classes = all_classes[:4]  # Take first 4
else:
    selected_classes = all_classes

print(f"\n✅ Selected classes for analysis: {selected_classes}")

---
## 📸 Part A: OpenCV Image Processing
### Task 1: Load Sample Images from Different Classes

In [ ]:
# Function to load sample images from each class
def load_sample_images(data_dir, classes, num_per_class=3):
    """
    Load sample images from each animal class
    
    Args:
        data_dir: Path to dataset directory
        classes: List of class names
        num_per_class: Number of images to load per class
    
    Returns:
        Dictionary with class names as keys and lists of images as values
    """
    sample_images = {}
    
    for cls in classes:
        class_path = os.path.join(data_dir, cls)
        image_files = [f for f in os.listdir(class_path) 
                       if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        
        # Load first 'num_per_class' images
        sample_images[cls] = []
        for img_file in image_files[:num_per_class]:
            img_path = os.path.join(class_path, img_file)
            # Read image using OpenCV (BGR format)
            img = cv2.imread(img_path)
            
            # Check if image loaded successfully
            if img is None:
                print(f"⚠️ Warning: Could not load {img_path}, skipping...")
                continue
            
            # Convert BGR to RGB for proper display
            try:
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                sample_images[cls].append(img_rgb)
            except Exception as e:
                print(f"⚠️ Warning: Could not convert {img_path}: {e}")
                continue
    
    return sample_images

# Load sample images
sample_images = load_sample_images(data_dir, selected_classes, num_per_class=3)

# Display loaded images
print(f"✅ Loaded {sum(len(imgs) for imgs in sample_images.values())} images total")
for cls, imgs in sample_images.items():
    print(f"  - {cls}: {len(imgs)} images")

In [ ]:
# Display sample images from each class
fig, axes = plt.subplots(len(selected_classes), 3, figsize=(12, 4*len(selected_classes)))
fig.suptitle('Sample Images from Each Animal Class', fontsize=16, fontweight='bold')

for i, (cls, imgs) in enumerate(sample_images.items()):
    for j, img in enumerate(imgs):
        # Handle both single class (1D array) and multiple classes (2D array)
        if len(selected_classes) == 1:
            ax = axes[j]
        else:
            ax = axes[i, j]
        
        ax.imshow(img)
        ax.axis('off')
        if j == 0:
            ax.set_title(f'{cls.upper()}', fontweight='bold', fontsize=12)

plt.tight_layout()
plt.savefig(f'{project_folder}/01_sample_images.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 01_sample_images.png")

### Task 2: Convert to Grayscale and Compare

In [ ]:
# Select one image from each class for detailed processing
selected_for_processing = {cls: imgs[0] for cls, imgs in sample_images.items()}

# Convert to grayscale
grayscale_images = {}
for cls, img in selected_for_processing.items():
    # Convert RGB to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    grayscale_images[cls] = gray

print("✅ Converted images to grayscale")

In [ ]:
# Display original vs grayscale comparison
fig, axes = plt.subplots(len(selected_classes), 2, figsize=(10, 5*len(selected_classes)))
fig.suptitle('Original vs Grayscale Comparison', fontsize=16, fontweight='bold')

for i, cls in enumerate(selected_classes):
    # Handle both single class (1D array) and multiple classes (2D array)
    if len(selected_classes) == 1:
        ax1 = axes[0]
        ax2 = axes[1]
    else:
        ax1 = axes[i, 0]
        ax2 = axes[i, 1]
    
    # Original image
    ax1.imshow(selected_for_processing[cls])
    ax1.set_title(f'{cls} - Original (RGB)', fontweight='bold')
    ax1.axis('off')
    
    # Grayscale image
    ax2.imshow(grayscale_images[cls], cmap='gray')
    ax2.set_title(f'{cls} - Grayscale', fontweight='bold')
    ax2.axis('off')

plt.tight_layout()
plt.savefig(f'{project_folder}/02_grayscale_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 02_grayscale_comparison.png")

### Task 3: Apply Gaussian and Median Filtering

In [ ]:
# Apply filters to grayscale images
gaussian_filtered = {}
median_filtered = {}

for cls, gray_img in grayscale_images.items():
    # Gaussian Blur: Smooths image using weighted average (good for reducing noise)
    # Kernel size (5, 5) - must be odd numbers
    # sigmaX=0 means calculate automatically
    gaussian = cv2.GaussianBlur(gray_img, (5, 5), 0)
    gaussian_filtered[cls] = gaussian
    
    # Median Filter: Replaces each pixel with median of neighboring pixels
    # Kernel size 5 - good for removing salt-and-pepper noise
    median = cv2.medianBlur(gray_img, 5)
    median_filtered[cls] = median

print("✅ Applied Gaussian and Median filters")
print("\n📝 Filter Details:")
print("  - Gaussian Blur: Kernel size (5x5), reduces Gaussian noise")
print("  - Median Filter: Kernel size 5, removes salt-and-pepper noise")

In [ ]:
# Display comparison: Original, Gaussian, Median
fig, axes = plt.subplots(len(selected_classes), 3, figsize=(15, 5*len(selected_classes)))
fig.suptitle('Filter Comparison: Original vs Gaussian vs Median', fontsize=16, fontweight='bold')

for i, cls in enumerate(selected_classes):
    # Handle both single class (1D array) and multiple classes (2D array)
    if len(selected_classes) == 1:
        ax1 = axes[0]
        ax2 = axes[1]
        ax3 = axes[2]
    else:
        ax1 = axes[i, 0]
        ax2 = axes[i, 1]
        ax3 = axes[i, 2]
    
    # Original grayscale
    ax1.imshow(grayscale_images[cls], cmap='gray')
    ax1.set_title(f'{cls} - Original', fontweight='bold')
    ax1.axis('off')
    
    # Gaussian filtered
    ax2.imshow(gaussian_filtered[cls], cmap='gray')
    ax2.set_title(f'{cls} - Gaussian Blur', fontweight='bold')
    ax2.axis('off')
    
    # Median filtered
    ax3.imshow(median_filtered[cls], cmap='gray')
    ax3.set_title(f'{cls} - Median Filter', fontweight='bold')
    ax3.axis('off')

plt.tight_layout()
plt.savefig(f'{project_folder}/03_filter_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 03_filter_comparison.png")

### Task 4: Edge Detection - Canny and Sobel Operators

In [ ]:
# Apply edge detection algorithms
canny_edges = {}
sobel_edges = {}

for cls, gray_img in grayscale_images.items():
    # Canny Edge Detection
    # Parameters: image, threshold1 (lower), threshold2 (upper)
    # If gradient > threshold2 → strong edge
    # If gradient < threshold1 → not an edge
    # In between → weak edge (kept if connected to strong edge)
    canny = cv2.Canny(gray_img, threshold1=50, threshold2=150)
    canny_edges[cls] = canny
    
    # Sobel Edge Detection
    # Detects edges by calculating gradient in X and Y directions
    # cv2.CV_64F: Output data type (64-bit float for accuracy)
    # dx=1, dy=0: Gradient in X direction (vertical edges)
    sobel_x = cv2.Sobel(gray_img, cv2.CV_64F, 1, 0, ksize=3)
    # dx=0, dy=1: Gradient in Y direction (horizontal edges)
    sobel_y = cv2.Sobel(gray_img, cv2.CV_64F, 0, 1, ksize=3)
    
    # Combine X and Y gradients to get magnitude
    sobel_combined = np.sqrt(sobel_x**2 + sobel_y**2)
    
    # Normalize to 0-255 range for display (with safety check for uniform images)
    max_val = np.max(sobel_combined)
    if max_val > 0:
        sobel_combined = np.uint8(255 * sobel_combined / max_val)
    else:
        sobel_combined = np.uint8(sobel_combined)
    
    sobel_edges[cls] = sobel_combined

print("✅ Applied Canny and Sobel edge detection")
print("\n📝 Edge Detection Details:")
print("  - Canny: Multi-stage algorithm, detects edges with precision")
print("  - Sobel: Gradient-based, detects edges using derivatives")

In [ ]:
# Display edge detection comparison
fig, axes = plt.subplots(len(selected_classes), 3, figsize=(15, 5*len(selected_classes)))
fig.suptitle('Edge Detection: Original vs Canny vs Sobel', fontsize=16, fontweight='bold')

for i, cls in enumerate(selected_classes):
    # Handle both single class (1D array) and multiple classes (2D array)
    if len(selected_classes) == 1:
        ax1 = axes[0]
        ax2 = axes[1]
        ax3 = axes[2]
    else:
        ax1 = axes[i, 0]
        ax2 = axes[i, 1]
        ax3 = axes[i, 2]
    
    # Original grayscale
    ax1.imshow(grayscale_images[cls], cmap='gray')
    ax1.set_title(f'{cls} - Original', fontweight='bold')
    ax1.axis('off')
    
    # Canny edges
    ax2.imshow(canny_edges[cls], cmap='gray')
    ax2.set_title(f'{cls} - Canny Edges', fontweight='bold')
    ax2.axis('off')
    
    # Sobel edges
    ax3.imshow(sobel_edges[cls], cmap='gray')
    ax3.set_title(f'{cls} - Sobel Edges', fontweight='bold')
    ax3.axis('off')

plt.tight_layout()
plt.savefig(f'{project_folder}/04_edge_detection.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 04_edge_detection.png")

### Task 5: Detect and Highlight Contours

In [ ]:
# Find and draw contours on images
contour_images = {}
contour_counts = {}

for cls in selected_classes:
    # Use Canny edges for contour detection
    edges = canny_edges[cls]
    
    # Find contours
    # cv2.RETR_EXTERNAL: Retrieve only external (outermost) contours
    # cv2.CHAIN_APPROX_SIMPLE: Compress horizontal/vertical segments
    contours, hierarchy = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a copy of original image to draw contours on
    img_with_contours = selected_for_processing[cls].copy()
    
    # Filter contours by area to remove small noise
    min_contour_area = 500  # Minimum area threshold
    significant_contours = [cnt for cnt in contours if cv2.contourArea(cnt) > min_contour_area]
    
    # Draw contours on image
    # -1 means draw all contours
    # (0, 255, 0) is green color in RGB
    # 2 is line thickness
    cv2.drawContours(img_with_contours, significant_contours, -1, (0, 255, 0), 2)
    
    contour_images[cls] = img_with_contours
    contour_counts[cls] = len(significant_contours)

print("✅ Detected and highlighted contours")
print("\n📊 Contour Counts:")
for cls, count in contour_counts.items():
    print(f"  - {cls}: {count} significant contours detected")

In [ ]:
# Display images with contours highlighted
fig, axes = plt.subplots(len(selected_classes), 2, figsize=(12, 6*len(selected_classes)))
fig.suptitle('Contour Detection on Animal Shapes', fontsize=16, fontweight='bold')

for i, cls in enumerate(selected_classes):
    # Handle both single class (1D array) and multiple classes (2D array)
    if len(selected_classes) == 1:
        ax1 = axes[0]
        ax2 = axes[1]
    else:
        ax1 = axes[i, 0]
        ax2 = axes[i, 1]
    
    # Original image
    ax1.imshow(selected_for_processing[cls])
    ax1.set_title(f'{cls} - Original', fontweight='bold')
    ax1.axis('off')
    
    # Image with contours
    ax2.imshow(contour_images[cls])
    ax2.set_title(f'{cls} - Contours ({contour_counts[cls]} detected)', fontweight='bold')
    ax2.axis('off')

plt.tight_layout()
plt.savefig(f'{project_folder}/05_contour_detection.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 05_contour_detection.png")

### Task 6: Analysis - Image Clarity and Edge Detection

#### 📝 Observations on Image Processing:

**1. Grayscale Conversion:**
- Reduces image complexity from 3 channels (RGB) to 1 channel (intensity)
- Makes edge detection more efficient and accurate
- Preserves structural information while removing color information

**2. Filtering Effects:**
- **Gaussian Blur:** Smooths images, reduces high-frequency noise, creates softer edges
- **Median Filter:** Better at preserving edges while removing noise, especially salt-and-pepper noise
- Both filters help improve edge detection by reducing false edges from noise

**3. Edge Detection Comparison:**
- **Canny:** More precise, detects thin edges, good for clean boundaries
- **Sobel:** Captures gradient magnitude, shows edge strength, more tolerant to noise

**4. Contours and Animal Shapes:**
- Contours successfully outline the main animal bodies
- Clear boundaries (like elephant ears, zebra body) produce strong contours
- Complex backgrounds may create additional contours
- Filtering before edge detection improves contour quality

**5. Relationship Between Image Clarity and Edges:**
- Sharper images → clearer edges → better contour detection
- High contrast between animal and background → stronger edge signals
- Animal features (stripes, ears, tusks) create distinctive edge patterns
- Blurred or low-contrast areas result in weak or missing edges

---

---
## 🤖 Part B: Deep Learning Classification
### Task 7: Prepare Dataset - Resize and Normalize

In [ ]:
# Create a subset dataset with selected classes
subset_dir = '/content/dataset_subset'
os.makedirs(subset_dir, exist_ok=True)

# Parameters
MAX_IMAGES_PER_CLASS = 150  # Limit images per class for faster training
IMG_SIZE = (224, 224)  # Standard size for MobileNetV2

print("📦 Creating dataset subset...")
for cls in selected_classes:
    src_dir = os.path.join(data_dir, cls)
    dst_dir = os.path.join(subset_dir, cls)
    os.makedirs(dst_dir, exist_ok=True)
    
    # Get all image files
    image_files = [f for f in os.listdir(src_dir) 
                   if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    # Copy limited number of images
    for img_file in image_files[:MAX_IMAGES_PER_CLASS]:
        src_path = os.path.join(src_dir, img_file)
        dst_path = os.path.join(dst_dir, img_file)
        shutil.copy2(src_path, dst_path)
    
    print(f"  ✅ {cls}: {len(os.listdir(dst_dir))} images")

print(f"\n✅ Dataset subset created at: {subset_dir}")

In [ ]:
# Create train and validation splits using Keras utilities
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
VALIDATION_SPLIT = 0.2  # 80% train, 20% validation

# Training dataset
# image_dataset_from_directory automatically:
# - Loads images from subdirectories (each subdirectory = one class)
# - Resizes images to specified size
# - Creates batches
# - Shuffles data
train_ds = tf.keras.utils.image_dataset_from_directory(
    subset_dir,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=123,  # For reproducibility
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Validation dataset
val_ds = tf.keras.utils.image_dataset_from_directory(
    subset_dir,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=123,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE
)

# Get class names
class_names = train_ds.class_names
num_classes = len(class_names)

print(f"✅ Dataset prepared:")
print(f"  - Classes: {class_names}")
print(f"  - Number of classes: {num_classes}")
print(f"  - Training batches: {len(train_ds)}")
print(f"  - Validation batches: {len(val_ds)}")
print(f"  - Image size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"  - Batch size: {BATCH_SIZE}")

In [ ]:
# Normalize images to [0, 1] range
# MobileNetV2 expects pixel values in [0, 1]
normalization_layer = layers.Rescaling(1./255)

# Apply normalization to datasets
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y))

# Optimize performance with caching and prefetching
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("✅ Images normalized to [0, 1] range")
print("✅ Dataset optimized with caching and prefetching")

### Task 8-9: Load Pretrained MobileNetV2 and Fine-tune

In [ ]:
# Load pretrained MobileNetV2 model
# weights='imagenet': Use weights pretrained on ImageNet dataset
# include_top=False: Exclude the final classification layer (we'll add our own)
# input_shape: Size of input images
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),
    include_top=False,
    weights='imagenet'
)

# Freeze the base model layers
# This prevents the pretrained weights from being updated during training
# We only train the new layers we add on top
base_model.trainable = False

print("✅ Loaded pretrained MobileNetV2")
print(f"  - Total layers in base model: {len(base_model.layers)}")
print(f"  - Base model trainable: {base_model.trainable}")
print(f"  - Input shape: {IMG_HEIGHT}x{IMG_WIDTH}x3")

In [ ]:
# Build the complete model
# Add custom layers on top of MobileNetV2
model = tf.keras.Sequential([
    base_model,  # Pretrained MobileNetV2
    
    # Global Average Pooling: Converts 2D feature maps to 1D feature vector
    # Reduces spatial dimensions while preserving important features
    layers.GlobalAveragePooling2D(),
    
    # Dropout: Randomly drops 20% of neurons during training
    # Helps prevent overfitting
    layers.Dropout(0.2),
    
    # Dense layer: Final classification layer
    # num_classes outputs (one for each animal class)
    # softmax activation: Converts outputs to probabilities that sum to 1
    layers.Dense(num_classes, activation='softmax')
])

print("✅ Model architecture created")
print("\n📊 Model Summary:")
model.summary()

In [ ]:
# Compile the model
# Optimizer: Adam - adaptive learning rate optimizer
# Loss: Sparse Categorical Crossentropy - for multi-class classification
# Metrics: Accuracy - track classification accuracy
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=['accuracy']
)

print("✅ Model compiled and ready for training")

### Task 10: Train Model (5-10 Epochs)

In [ ]:
# Train the model
EPOCHS = 10  # Number of times to iterate over the entire dataset

print("🚀 Starting training...")
print(f"  - Epochs: {EPOCHS}")
print(f"  - Training samples per epoch: ~{len(train_ds) * BATCH_SIZE}")
print(f"  - Validation samples per epoch: ~{len(val_ds) * BATCH_SIZE}")
print("\n" + "="*60)

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    verbose=1  # Show progress bar
)

print("\n" + "="*60)
print("✅ Training completed!")

In [ ]:
# Plot training history
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

# Plot accuracy
ax1.plot(history.history['accuracy'], label='Training Accuracy', marker='o')
ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', marker='s')
ax1.set_title('Model Accuracy over Epochs', fontweight='bold', fontsize=14)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Accuracy')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot loss
ax2.plot(history.history['loss'], label='Training Loss', marker='o')
ax2.plot(history.history['val_loss'], label='Validation Loss', marker='s')
ax2.set_title('Model Loss over Epochs', fontweight='bold', fontsize=14)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Loss')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{project_folder}/06_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

# Print final metrics
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
final_train_loss = history.history['loss'][-1]
final_val_loss = history.history['val_loss'][-1]

print("\n📊 Final Training Metrics:")
print(f"  - Training Accuracy: {final_train_acc:.4f} ({final_train_acc*100:.2f}%)")
print(f"  - Validation Accuracy: {final_val_acc:.4f} ({final_val_acc*100:.2f}%)")
print(f"  - Training Loss: {final_train_loss:.4f}")
print(f"  - Validation Loss: {final_val_loss:.4f}")
print("\n💾 Saved to Google Drive: 06_training_history.png")

### Task 11: Confusion Matrix and Sample Predictions

In [ ]:
# Make predictions on validation set
print("🔮 Generating predictions on validation set...")

# Get all validation data
y_true = []
y_pred = []
val_images = []

for images, labels in val_ds:
    predictions = model.predict(images, verbose=0)
    y_true.extend(labels.numpy())
    y_pred.extend(np.argmax(predictions, axis=1))
    val_images.extend(images.numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)
val_images = np.array(val_images)

print(f"✅ Generated predictions for {len(y_true)} validation images")

In [ ]:
# Create confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=class_names, yticklabels=class_names,
            cbar_kws={'label': 'Count'})
plt.title('Confusion Matrix - Animal Classification', fontweight='bold', fontsize=16)
plt.ylabel('True Label', fontweight='bold')
plt.xlabel('Predicted Label', fontweight='bold')
plt.tight_layout()
plt.savefig(f'{project_folder}/07_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 07_confusion_matrix.png")

In [ ]:
# Print classification report
print("\n📊 Classification Report:")
print("="*60)
print(classification_report(y_true, y_pred, target_names=class_names))

# Calculate per-class accuracy
print("\n📈 Per-Class Accuracy:")
print("="*60)
for i, class_name in enumerate(class_names):
    class_mask = y_true == i
    num_samples = np.sum(class_mask)
    
    # Check if there are samples for this class
    if num_samples > 0:
        class_accuracy = np.sum(y_pred[class_mask] == y_true[class_mask]) / num_samples
        print(f"  {class_name}: {class_accuracy:.4f} ({class_accuracy*100:.2f}%)")
    else:
        print(f"  {class_name}: No samples in validation set")

In [ ]:
# Display sample predictions
# Select 12 random samples
num_samples = min(12, len(val_images))
random_indices = np.random.choice(len(val_images), num_samples, replace=False)

fig, axes = plt.subplots(3, 4, figsize=(16, 12))
fig.suptitle('Sample Predictions on Validation Set', fontsize=16, fontweight='bold')

for idx, ax in enumerate(axes.flat):
    if idx < num_samples:
        img_idx = random_indices[idx]
        img = val_images[img_idx]
        true_label = class_names[y_true[img_idx]]
        pred_label = class_names[y_pred[img_idx]]
        
        # Determine if prediction is correct
        is_correct = y_true[img_idx] == y_pred[img_idx]
        color = 'green' if is_correct else 'red'
        
        ax.imshow(img)
        ax.set_title(f'True: {true_label}\nPred: {pred_label}', 
                     color=color, fontweight='bold')
        ax.axis('off')
    else:
        ax.axis('off')

plt.tight_layout()
plt.savefig(f'{project_folder}/08_sample_predictions.png', dpi=150, bbox_inches='tight')
plt.show()

print("💾 Saved to Google Drive: 08_sample_predictions.png")
print("\n🟢 Green = Correct Prediction")
print("🔴 Red = Incorrect Prediction")

### Task 12: Discussion - Accuracy Differences and Causes

In [ ]:
# Analyze misclassifications
misclassified_mask = y_true != y_pred
num_misclassified = np.sum(misclassified_mask)
total_samples = len(y_true)

print("🔍 Misclassification Analysis:")
print("="*60)
print(f"Total validation samples: {total_samples}")
print(f"Correctly classified: {total_samples - num_misclassified}")
print(f"Misclassified: {num_misclassified}")
print(f"Overall accuracy: {(total_samples - num_misclassified) / total_samples * 100:.2f}%")

# Find most common misclassifications
print("\n❌ Most Common Misclassifications:")
print("="*60)
misclass_pairs = {}
for true_idx, pred_idx in zip(y_true[misclassified_mask], y_pred[misclassified_mask]):
    pair = (class_names[true_idx], class_names[pred_idx])
    misclass_pairs[pair] = misclass_pairs.get(pair, 0) + 1

# Sort by frequency
sorted_misclass = sorted(misclass_pairs.items(), key=lambda x: x[1], reverse=True)
for (true_class, pred_class), count in sorted_misclass[:5]:
    print(f"  {true_class} → {pred_class}: {count} times")

#### 📝 Discussion: Accuracy Differences Across Classes

**Possible Causes of Accuracy Variations:**

**1. Visual Similarity Between Classes:**
- Some animals may share similar features (body shape, color, texture)
- Example: If buffalo and rhino have similar body structures, the model might confuse them
- Animals with distinctive features (zebra stripes, elephant trunk) are easier to classify

**2. Dataset Imbalance:**
- Classes with more training images generally achieve higher accuracy
- Uneven distribution of samples can bias the model toward majority classes
- Limited samples for some classes lead to underfitting

**3. Image Quality and Variety:**
- **Lighting conditions:** Dark or overexposed images are harder to classify
- **Viewing angles:** Side views vs. front views affect feature visibility
- **Occlusions:** Animals partially hidden by vegetation or other objects
- **Image resolution:** Blurry or low-resolution images lack discriminative features

**4. Background Complexity:**
- Cluttered backgrounds can confuse the model
- Animals in their natural habitat may blend with surroundings
- Simple backgrounds make animal features more prominent

**5. Pose and Position Variations:**
- Standing, sitting, or lying positions change appearance significantly
- Multiple animals in one image can confuse the classifier
- Unusual poses not well-represented in training data lead to errors

**6. Model Limitations:**
- **Transfer learning constraints:** MobileNetV2 was pretrained on ImageNet, which may not perfectly align with wildlife images
- **Limited fine-tuning:** Only training top layers (base model frozen) restricts adaptation
- **Model capacity:** MobileNetV2 is lightweight - more complex models might achieve better accuracy

**7. Training Parameters:**
- **Epochs:** 10 epochs might be insufficient for full convergence
- **Learning rate:** Default Adam settings may not be optimal
- **Batch size:** Affects gradient updates and generalization

**Improvements to Consider:**
- Increase training data through data augmentation (rotation, flip, zoom)
- Balance dataset across all classes
- Unfreeze and fine-tune deeper layers of base model
- Train for more epochs with early stopping
- Experiment with different pretrained models (ResNet50, EfficientNet)
- Apply better preprocessing (background removal, cropping)

---

### Save the Trained Model

In [ ]:
# Save the model to Google Drive
model_save_path = f'{project_folder}/african_wildlife_model.keras'
model.save(model_save_path)

print(f"✅ Model saved to: {model_save_path}")
print("\n📦 You can download this model from your Google Drive later!")

---
## 🎉 Assignment Complete!

### Summary of Results:

**Part A - OpenCV Image Processing:**
✅ Loaded and processed images from multiple animal classes
✅ Applied grayscale conversion and compared with originals
✅ Implemented Gaussian and Median filtering
✅ Performed edge detection using Canny and Sobel operators
✅ Detected and highlighted contours on animal shapes
✅ Analyzed relationship between image clarity and edge detection

**Part B - Deep Learning Classification:**
✅ Prepared dataset with resizing and normalization
✅ Used pretrained MobileNetV2 model
✅ Fine-tuned on selected animal categories
✅ Trained for 10 epochs with validation monitoring
✅ Generated confusion matrix and classification report
✅ Displayed sample predictions with accuracy analysis
✅ Discussed accuracy differences and possible causes

### 📁 All Results Saved to Google Drive:
Check your Google Drive folder: `MyDrive/african_wildlife_project/`

---

### 🚀 Next Steps (Optional):
1. Try unfreezing some base model layers and fine-tuning further
2. Implement data augmentation for better generalization
3. Experiment with ResNet50 for comparison
4. Increase dataset size for improved accuracy

**Happy Learning! 🎓**